## Numerical simulation to obtain lattice total rebinding probability at various $k_a/k_D$ ratios.

In [1]:
import sys
import math
import numpy as np
from numpy import pi, sqrt, inf, log
from ecell4 import *

L = 100
voxel_radius = 0.005
D = 1
rng = GSLRandomNumberGenerator()
rng.seed(1)
kD = 4 * pi * (2 * voxel_radius) * D
dt = (4 * voxel_radius * voxel_radius) / (6 * D)
p=1
w = spatiocyte.create_spatiocyte_world_cell_list_impl(ones() * L, voxel_radius, Integer3(3, 3, 3), rng)

def singlerunA(dt, Pacc,alpha): #for evaluating G_n
    assert Pacc*alpha  <= 1
    assert alpha<=1
    coord1 = w.position2coordinate(ones() * L * 0.5)
    coord2 = w.get_neighbor(coord1, 0)
    dt = dt*alpha
    t, nsteps = 0.0, 0
    while all(0 <= dim < L for dim in w.coordinate2position(coord2)) and nsteps < maxstep:
        rnd = w.rng().uniform_int(0, 11)
        newcoord = w.get_neighbor(coord2, rnd)
        if newcoord == coord1:
            if Pacc * alpha >= w.rng().uniform(0, 1):
                return nsteps+1
        elif alpha >= w.rng().uniform(0, 1):
            coord2 = newcoord
            nsteps += 1
        t += dt      
    return -1

def singlerunB(dt, Pacc): #for evaluating H_n
    assert Pacc<= 1
    coord1 = w.position2coordinate(ones() * L * 0.5)
    coord2 = w.get_neighbor(coord1, 0)
    dt = dt
    t, nsteps = 0.0, 0
    while all(0 <= dim < L for dim in w.coordinate2position(coord2)) and nsteps < maxstep:
        rnd = w.rng().uniform_int(0, 11)    
        newcoord = w.get_neighbor(coord2, rnd)
        if newcoord == coord1:
            if Pacc >= w.rng().uniform(0, 1):
                return nsteps+1
        else:
            coord2 = newcoord
        t += dt
        nsteps += 1
    return -1

## set ka/kD ratio and number of simulation runs

In [2]:
ka_factor = 100 #ka/kD ratio
ntrials = 1000# #number of runs, int(1e9) used in the paper

ka = kD * ka_factor
r0=0.256318
Pacc = (1/r0-1)*ka_factor
maxstep= 10 #maximum number of steps
if Pacc>1:
    res = [singlerunA(dt, Pacc,p/Pacc) for _ in range(ntrials)]
else:    
    res = [singlerunB(dt, Pacc) for _ in range(ntrials)]
print('ka/kD ratio',ka_factor)    
print('rebinding probability until n='+str(maxstep)+', with '+str(ntrials)+' runs: \nP=',1-res.count(-1)/ntrials)

ka/kD ratio 100
rebinding probability until n=10, with 1000 runs: 
P= 0.992
